In [1]:
import pandas as pd
import numpy as np
#get the 'df' row where the index contains 'string'
#PARAMS
#df: dataframe
#string: string
def get_row(df, string):
    return pd.Series(df.index).str.contains(string.strip(), case=False)

#get the 'df' column where the index contains 'string'
#PARAMS
#df: dataframe
#string: string
def get_column(df, string):
    return pd.Series(df.columns).str.contains(string.strip(), case=False)

#calculate the difference in minutes between two given times expressed as HH:MM
#PARAMS
#time_table: list
#day: string
#index, index2: integer
def difference(time_table, day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

#get the name expressed in the index of dataframe 'dft' that contains 'nome'
#PARAMS
#dft: dataframe
#nome: string
def getName(df, nome):
    if nome.lower() == "crema":
        nome ="crema,"
    city = df[get_row(df, nome).values].index[0]
    return city

#returns a pd.Series where keys are pair of destinations' names and the value is the time to go
#from the first city to the second
#PARAMS
#schedule: lists of cities
#dframe: dataframe
#RETURNS: city1 $ city2 -> time
def new_route(schedule, dframe):
    times = []
    keys = []
    
    row = dframe[get_row(dframe, schedule[0]).values]
    key1 = ''
    key2 = ''
    
    for i in range(1,len(schedule)):
        key1 = row.index[0]
        paese=schedule[i]
        paese=paese.strip()
        if 'Martignana' in paese:
            paese = 'Martignana di Po'
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        try:
            value=row.loc[:,get_column(dframe, paese).values].values[0][0]
        except:
            print(row.loc[:,get_column(dframe, paese).values])
            print(paese)
            return
        times.append(round(value))
        row = dframe[get_row(dframe, paese).values]
        try:
            key2 = row.index[0]
        except:
            raise Exception("ciao")
        keys.append(key1 + ' $ ' + key2)
    steps_times = pd.Series(times , keys)
    return steps_times

#returns a list of cities from a pd.Series in the format obtained by 'new_route'
#PARAMS
#serie: pd.Series
def cities_list(serie):
    cities = []
    keys = serie.keys()
    for i in range(0, len(serie)):
        splitted = keys[i].split("$")
        cities = cities + [splitted[0].strip()]
    cities = cities + [splitted[1].strip()]
    return cities 

#returns the distances expressed in dft from 'start' to 'end'
#PARAMS
#start, end: string
#dft: dataframe
def timeDistance(start, end, dft):
    start_city_row = dft[get_row(dft, start.strip()).values]
    return start_city_row.loc[:,get_column(dft, end).values].values[0][0]

#given a city 'to_insert' returns the pd.Serie where the keys are pair of cities and values are the time to go
#from the first city to 'to_insert' plus 'to_insert' to second city
#PARAMS
#steps_times: pd.Series (new_route output)
#to_insert: string
#dframe: dataframe
def find_best_position(steps_times, to_insert, dframe):
    times = []
    key2 = []
    for i in range(len(steps_times)):
        cities = steps_times.keys()[i].split('$')
        if(cities[0].strip()=="San Bassano, Mons. Frosi (ex piazza Scuole)"):
            row1 = dframe[get_row(dframe, 'San Bassano, Mons. Frosi').values]
        else:
            row1 = dframe[get_row(dframe, cities[0].strip()).values]
        f_c1_to_curr = row1.loc[:,get_column(dframe, to_insert).values].values[0][0]
        row2 = dframe[get_row(dframe, to_insert).values]
        f_curr_to_c2 = row2.loc[:,get_column(dframe, cities[1].strip()).values].values[0][0]
        times.append(f_c1_to_curr + f_curr_to_c2)
    s3 = pd.Series(times, steps_times.keys())
    return s3

#returns the index in lista where to put 'to_insert' using find_best_position
#PARAMS
#lista: list
#schedule: pd.Series
#to_insert: string
#dframe: dataframe
def best_index(lista, schedule, to_insert, dframe):
    idx = find_best_position(schedule, to_insert, dframe).idxmin()
    cities = idx.split("$")
    a=cities[0].strip()
    new_lista = [obj.lower() for obj in lista]
    try:
        #print("classic return")
        return new_lista.index(a.lower())+1
    except:
        #problema nome diverso
        print("Nome strano", a, sep="  ")
        if(a.lower() == 'casale cremasco'):
            a = 'Casale Cremasco-Vidolasco, Roma, 56'
            return new_lista.index(a)+1
        elif(a.lower() == 'gadesco'):
            a = 'Gadesco-Pieve Delmona, Giovanni Lonati 9'
            return new_lista.index(a)+1
        elif(a=="San Bassano, Mons. Frosi (ex piazza Scuole)"):
            a = "San Bassano, Mons. Frosi"
            return new_lista.index(a)+1